The code which contains the main analysis method. Any of the data examples can be run through the analysis method to see the individual graphs.

In [84]:
import ccxt
import pandas as pd
import statistics as stat
from math import pi
import time
import random
import os
import numpy as np

#plotting
from bokeh.plotting import figure, show, output_file
from bokeh.io import export_svgs
from bokeh.io import output_notebook
from bokeh.models import Axis, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.plotting import figure

In [3]:
'''
Things I don't like:
- in analyse_symbol() peak methods create rolling averages, and they are dependent on being called in a certain order
- too much dependence on the specific name of the rolling average columns in the DFs (in peak methods and plot_pumps)
    - tho these names are auto generated so maybe not a huge deal
- there are a lot of DF's to keep track of
- naming is passable, but not always consistent
'''

'''
MAIN PROBLEM : Too much is dependent on the hardcoded column names, things should be restructured and cleaned up a lot
Needs to be updated to use a proper structure and file system, to make it usable to others.
'''

# gets symbol name from the csv file
def get_symbol(f_path):
    df = pd.read_csv(f_path,index_col=0,parse_dates=["Timestamp"])
    filename = os.path.basename(f_path)
    symbol_name = filename.split("_")[1].replace("-","/")
    print("Loading:",symbol_name)
    return symbol_name

# extracts the symbol pairs and stores them in a df per exchange
def extract_symbol_df_from_csvs(folder):
    for subdir, dirs, files in os.walk(folder):
        symbols = []
        
        exchange_n = subdir.split("/")[-1]
        
        if 'data' not in exchange_n:
            for file in files:
                if ".csv" in file:
                    symbols.append(get_symbol('../data/'+subdir+'/'+file))
            header = ['Symbol']
            df = pd.DataFrame(symbols, columns=header)
            filename = '{}_symbols.csv'.format(exchange_n)        
            df.to_csv(filename)

# analyses all the symbol pairs in subfolders of a given folder
# returns a df indexed by exchange with the number price and volume spikes, and number of pumps
def analyse_folder(folder,v_thresh,p_thresh,win_size=24,c_size='1h'):
    row_list = [] # list for each row of the result df
    
    # -- loop through folders --
    for subdir, dirs, files in os.walk(folder):
        for file in files:
                if ".csv" in file:
                    f_path = subdir+'/'+file
                    result_row = analyse_symbol(f_path,v_thresh,p_thresh,win_size,c_size)
                    row_list.append(result_row)
   
    # -- create result df --
    df = pd.DataFrame(row_list)
    df.set_index('Exchange',inplace=True)
    df.sort_index(inplace=True)
                    
    return df

# returns final dataframe
def analyse_symbol(f_path,v_thresh,p_thresh,win_size=24,c_size='1h',plot=False):  
    '''
    USAGE:
    f_path : path to OHLCV csv e.g.'../data/binance/binance_STORJ-BTC_[2018-04-20 00.00.00]-TO-[2018-05-09 23.00.00].csv'
    v_thresh : volume threshold e.g. 5 (500%)
    p_thresh : price threshold e.g. 1.05 (5%)
    c_size : candle size
    win_size : size of the window for the rolling average, in hours
    '''
    # -- load the data --
    exchange_name,symbol_name,df = load_csv(f_path)
    
    # -- find spikes --
    vmask,vdf = find_vol_spikes(df,v_thresh,win_size)
    num_v_spikes = get_num_rows(vdf) # the number of volume spikes found for this symbol pair
    
    pmask,pdf = find_price_spikes(df,p_thresh,win_size)
    num_p_spikes = get_num_rows(pdf)
    
    pdmask,pddf = find_price_dumps(df,win_size)
    
    vdmask,vddf = find_volume_dumps(df,win_size)
    
    # find coinciding price and volume spikes
    vp_combined_mask = (vmask) & (pmask)
    vp_combined_df = df[vp_combined_mask]
    num_vp_combined_rows = get_num_rows(vp_combined_df)
    
    # coinciding price and volume spikes for alleged P&D (more than 1x per given time removed)
    vp_combined_rm = rm_same_day_pumps(vp_combined_df)
    num_alleged = get_num_rows(vp_combined_rm)
    
    # find coniciding price and volume spikes with dumps afterwards
    ''' at some point should probably be renamed '''
    final_combined_mask = (vmask) & (pmask) & (pdmask)
    final_combined = df[final_combined_mask]
    final_combined_rm = rm_same_day_pumps(final_combined) # remove indicators which occur on the same day
    num_final_combined = get_num_rows(final_combined_rm)
    
    # -- plot --
    if plot is True:
        plot_pumps(symbol_name,exchange_name,win_size,df,pdf,vdf,vp_combined_df,pddf,final_combined,final_combined_rm,
                   plot_pRA=True,plot_ppeaks=True,plot_vRA=True,plot_vpeaks=True)
    
    row_entry = {'Exchange':exchange_name,
                 'Symbol':symbol_name,
                 'Price Spikes':num_p_spikes,
                 'Volume Spikes':num_v_spikes,
                 'Alleged Pump and Dumps':num_alleged,
                 'Pump and Dumps':num_final_combined}

    return row_entry

def get_num_rows(df):
    return df.shape[0]

def rm_same_day_pumps(df):
    # Removes spikes that occur on the same day
    df = df.copy()
    df['Timestamp_DAYS'] = df['Timestamp'].apply(lambda x: x.replace(hour=0, minute=0, second=0))
    df = df.drop_duplicates(subset='Timestamp_DAYS', keep='last')
    return df

# finds volume spikes in a given df, with a certain threshold and window size
# returns a (boolean_mask,dataframe) tuple
def find_vol_spikes(df,v_thresh,win_size):
    # -- add rolling average column to df --
    vRA = str(win_size)+'h Volume RA'
    add_RA(df,win_size,'Volume',vRA)
    
    # -- find spikes -- 
    vol_threshold = v_thresh*df[vRA] # v_thresh increase in volume
    vol_spike_mask = df["Volume"] > vol_threshold # where the volume is at least v_thresh greater than the x-hr RA
    df_vol_spike = df[vol_spike_mask]
    
    return (vol_spike_mask,df_vol_spike)

# finds price spikes in a given df, with a certain threshold and window size
# returns a (boolean_mask,dataframe) tuple
def find_price_spikes(df,p_thresh,win_size):
    # -- add rolling average column to df --
    pRA = str(win_size)+'h Close Price RA'
    add_RA(df,win_size,'Close',pRA)
    
    # -- find spikes -- 
    p_threshold = p_thresh*df[pRA] # p_thresh increase in price
    p_spike_mask = df["High"] > p_threshold # where the high is at least p_thresh greater than the x-hr RA
    df_price_spike = df[p_spike_mask]
    return (p_spike_mask,df_price_spike)


# finds price dumps in a given df, with a certain threshold and window size
# requires a price rolling average column of the proper window size and naming convention
# returns a (boolean_mask,dataframe) tuple
def find_price_dumps(df,win_size):
    pRA = str(win_size)+"h Close Price RA"
    pRA_plus = pRA + "+" + str(win_size)
    
    df[pRA_plus] = df[pRA].shift(-win_size)
    price_dump_mask = df[pRA_plus] <= (df[pRA] + df[pRA].std())
    # if the xhour RA from after the pump was detected is <= the xhour RA (+std dev) from before the pump was detected
    # if the price goes from the high to within a range of what it was before
    
    df_p_dumps = df[price_dump_mask]
    return (price_dump_mask,df_p_dumps)

def find_volume_dumps(df,win_size):
    vRA = str(win_size)+"h Volume RA"
    vRA_plus = vRA + "+" + str(win_size)
    
    df[vRA_plus] = df[vRA].shift(-win_size)
    price_dump_mask = df[vRA_plus] <= (df[vRA] + df[vRA].std())
    # if the xhour RA from after the pump was detected is <= the xhour RA (+std dev) from before the pump was detected
    # if the volume goes from the high to within a range of what it was before
    
    df_p_dumps = df[price_dump_mask]
    return (price_dump_mask,df_p_dumps)
    
# adds a rolling average column with specified window size to a given df and col
def add_RA(df,win_size,col,name):
    df[name] = pd.Series.rolling(df[col],window=win_size,center=False).mean()

# returns a (exchange_name,symbol_name,dataframe) tuple
def load_csv(f_path,suppress=True):
    '''suppress : suppress output of the exchange and symbol name'''
    
    df = pd.read_csv(f_path,index_col=0,parse_dates=["Timestamp"])
    filename = os.path.basename(f_path)
    exchange_name = filename.split("_")[0]
    symbol_name = filename.split("_")[1].replace("-","/")
    
    if not suppress:
        print("Exchange:",exchange_name,"\nSymbol:",symbol_name)
    
    return (exchange_name,symbol_name,df)
    
    
# Uses a scatterplot to mark selected locations in a DF with an X
# Intended to be used to plot ontop of an existing figure
# e.g. plot_markers(base_plot,df,'Timestamp','Price')

def plot_markers(plot, df, xcol_name, ycol_name, color="red", marker="x",legend_name=None):
    markers = plot.scatter(df[xcol_name], df[ycol_name], color=color, marker=marker, legend=legend_name, 
                           muted_alpha = 0.5, muted_color=color)
    markers.glyph.size = 10
    markers.glyph.line_width = 2
    markers.level = 'overlay'
    return markers

In [92]:
# Main plotting method 
def plot_pumps(symbol_name,exchange_name,win_size,df,p_spike_df,v_spike_df,vp_combined_df,price_dump_df,final_df,final_df_rm,
                   plot_pRA=True,plot_ppeaks=True,plot_vRA=True,plot_vpeaks=True):
    '''
    Still needs support for different candle times (change w, not hard)
    '''
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

    # ---CANDLE PLOT---
    inc = df.Close > df.Open
    dec = df.Open > df.Close

    msec = 1000
    minute = 60 * msec
    hour = 60 * minute

    w = hour/2 # half an hour (should be half the candle size)

    p_candle = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, plot_height=450, 
                      title = symbol_name+" Candlestick"+" | Exchange: "+exchange_name, y_axis_label='Price')
    p_candle.xaxis.major_label_orientation = pi/4
    p_candle.grid.grid_line_alpha=0.3

    # turn off scientific notation for y axis
    yaxis = p_candle.select(dict(type=Axis, layout="left"))[0]
    yaxis.formatter.use_scientific = False

    # plot candles
    p_candle.segment(df.Timestamp, df.High, df.Timestamp, df.Low, color="black")
    p_candle.vbar(df.Timestamp[inc], w, df.Open[inc], df.Close[inc], fill_color="#40a075", line_color="black") # green
    p_candle.vbar(df.Timestamp[dec], w, df.Open[dec], df.Close[dec], fill_color="#F2583E", line_color="black") # red

    # marking peaks
    if plot_ppeaks:
        price_peaks = plot_markers(p_candle,p_spike_df,'Timestamp','High',legend_name='Price Increase',color="orange")
        combined_peaks = plot_markers(p_candle,vp_combined_df,'Timestamp','High',legend_name='Price + Volume Increase',color="brown")
        price_dumps = plot_markers(p_candle,final_df,'Timestamp','High',legend_name='Price + Volume Increase + Volume Decrease',color="red")
        final = plot_markers(p_candle,final_df_rm,'Timestamp','High',legend_name='Pump and Dump',color="blue",marker="diamond")

    # price rolling avg
    if plot_pRA:
        pRA = str(win_size)+"h Close Price RA"
        p_ra_leg = str(win_size)+"h Rolling Avg."
        p_candle.line(df.Timestamp, df[pRA], line_width=2, color="green",legend=p_ra_leg)


    # add mutable legend
    p_candle.legend.location = "top_right"
    p_candle.legend.click_policy= "mute"


    # ---VOLUME PLOT---
    # create a new plot with a title and axis labels
    p_vol = figure(tools=TOOLS, x_axis_label='Date', y_axis_label='Volume',
                   x_axis_type="datetime",x_range=p_candle.x_range, plot_width=1000, plot_height=200)

    vol_yaxis = p_vol.select(dict(type=Axis, layout="left"))[0]
    vol_yaxis.formatter.use_scientific = False

    # plot volume
    p_vol.line(df.Timestamp, df.Volume, line_width=2)

    # marking peaks
    if plot_vpeaks:
        vol_peaks = plot_markers(p_vol,v_spike_df,'Timestamp','Volume',legend_name='Volume Increase',color="purple")
        combined_vol_peaks = plot_markers(p_vol,vp_combined_df,'Timestamp','Volume',legend_name='Price + Volume Increase',color="magenta")
        combined_dump_vol = plot_markers(p_vol,final_df,'Timestamp','Volume',legend_name='Price + Volume Increase + Volume Decrease',color="red")

    # rolling avg
    if plot_vRA:
        vRA = str(win_size)+"h Volume RA"
        v_ra_leg = str(win_size)+"h Rolling Avg."
        p_vol.line(df.Timestamp, df[vRA], line_width=2, color="green",legend=v_ra_leg)

    # add mutable legend
    p_vol.legend.location = "top_right"
    p_vol.legend.click_policy= "mute"
    
    # change num ticks
    p_candle.xaxis[0].ticker.desired_num_ticks = 20
    p_vol.xaxis[0].ticker.desired_num_ticks = 20


    # ---COMBINED PLOT---
    p = gridplot([[p_candle],[p_vol]])

    output_notebook()
    p_candle.output_backend = "svg"
    p_vol.output_backend = "svg"
    show(p)

In [50]:
exc,sn,tdf = load_csv('../revision_data/binance/binance_WABI-BTC_[2018-08-19 00:00:00]-TO-[2018-09-06 12:00:00].csv')

In [51]:
tdf = tdf[(tdf['Timestamp'] > '2018-08-19 00:00:00') & (tdf['Timestamp'] < '2018-08-22 09:00:00')]
tdf.to_csv('../revision_data/binance/binance_WABI-BTC_[2018-08-19 00:00:00]-TO-[2018-08-22 09:00:00].csv')

# Main analysis method in the following block

In [93]:
# MAIN ANALYSIS BLOCK, any of the data will work if plugged into the file path,
# the basic parameters can also be changed here
analyse_symbol(f_path='../data/lbank/lbank_DBC-NEO_[2018-05-16 10.00.00]-TO-[2018-04-26 11.00.00].csv',
               v_thresh = 4,
               p_thresh = 1.05,
               win_size = 12,
               c_size = '1h',
               plot = True)

Loading BokehJS ...

{'Alleged Pump and Dumps': 15,
 'Exchange': 'lbank',
 'Price Spikes': 114,
 'Pump and Dumps': 13,
 'Symbol': 'DBC/NEO',
 'Volume Spikes': 45}

In [446]:
result = analyse_folder(folder='../data',
               v_thresh = 4,
               p_thresh = 1.05,
               win_size = 12)

In [ ]:
#preps result DF for further breakdown
add_crypto_pair_col(result,fiat)
add_market_cap(result)
add_low_mc_percent(result)

result = result[['Symbol','Crypto Pair','Price Spikes','Volume Spikes','Alleged Pump and Dumps','Pump and Dumps','Low Market Cap']]
result.head()

In [449]:
# exchange breakdown, meant to be run after using 'result = analyse_folder(...)'

# number of pumps
npumps = result['Pump and Dumps'].sum()
a_npumps = result['Alleged Pump and Dumps'].sum() # alleged pumps
true_pump_rate = (npumps/a_npumps) * 100

nsyms = result.shape[0]
usyms = len(result['Symbol'].unique())

# calculate crypto pair percentage
crypto_pairs = result[result['Crypto Pair'] == True]
overall_crypto_pair = (crypto_pairs.shape[0] / result.shape[0]) * 100
total_cp_pumps = crypto_pairs['Pump and Dumps'].sum()
total_cp_percent = (total_cp_pumps / npumps)*100

# calculate low market cap percentage
low_mc = result[result['Low Market Cap'] == True]
overall_low_mc = (low_mc.shape[0] / result.shape[0]) * 100
total_low_mc = low_mc['Pump and Dumps'].sum()
total_low_mc_percent = (total_low_mc / npumps)*100

print('Total number of ALLEGED pump and dumps:',a_npumps)
print('Total number of pump and dumps:',npumps)
print('True Pump Rate %:',true_pump_rate)
print('Crypto Pair %:',overall_crypto_pair,'| Crypto Pair Pump %:',total_cp_percent)
print('Low Market Cap %:',overall_low_mc,'| Low Market Cap Pump %:',total_low_mc_percent)
print('Total number of symbols:',nsyms)
print('Total unique symbols:',usyms)
print('Pumps per symbol',npumps/nsyms)

exchanges = ['binance','bittrex','kraken','kucoin','lbank']
syms_per = []

for e in exchanges:
    syms_per.append(result.loc[e,'Symbol'].shape[0])

# creates the main df broken down by exchange, with num. pumps column    
adf = get_all_pumps(result,exchanges,'Pump and Dumps') 

adf['Num. Symbols'] = syms_per
adf['% of Total Pumps'] = adf['Num. Pumps'] / npumps * 100
adf['% of Total Symbols'] = adf['Num. Symbols'] / nsyms * 100
adf = adf[['Num. Symbols','Num. Pumps','% of Total Symbols','% of Total Pumps']]
adf

Total number of ALLEGED pump and dumps: 2150
Total number of pump and dumps: 1617
True Pump Rate %: 75.20930232558139
Crypto Pair %: 96.3152507676561 | Crypto Pair Pump %: 96.96969696969697
Low Market Cap %: 75.43500511770726 | Low Market Cap Pump %: 81.75633889919605
Total number of symbols: 977
Total unique symbols: 737
Pumps per symbol 1.655066530194473


,Num. Symbols,Num. Pumps,% of Total Symbols,% of Total Pumps
Exchange,,,,
binance,302,536,30.910952,33.147805
bittrex,242,580,24.769703,35.868893
kraken,57,75,5.834186,4.638219
kucoin,295,293,30.194473,18.119975
lbank,81,133,8.290686,8.225108


In [455]:
results_filtered = result[(result['Crypto Pair'] == True) & (result['Low Market Cap'] == True)]

# FOR USING AFTER FILTERING THE DATA TO ONLY CRYPTO/CRYPTO PAIRS AND LOW MARKET CAP
def get_all_pumps_filtered(df,df2,exchanges,col_name):
    row_list = []
    for e in exchanges:
        total = get_pumps(df,e,col_name)
        total_f = get_pumps(df2,e,col_name)
        row_list.append({'Exchange':e,'Num. Pumps':total,'Num. Pumps (Filtered)':total_f})
        
    result = pd.DataFrame(row_list)
    result.set_index('Exchange',inplace=True)
    result.sort_index(inplace=True)
    return result

# exchange breakdown

# number of pumps
npumps = result['Pump and Dumps'].sum()
f_npumps = results_filtered['Pump and Dumps'].sum()
a_npumps = result['Alleged Pump and Dumps'].sum() # alleged pumps
f_a_npumps = results_filtered['Alleged Pump and Dumps'].sum() # alleged pumps
true_pump_rate = (f_npumps/a_npumps) * 100

nsyms = result.shape[0]
usyms = len(result['Symbol'].unique())
f_nsyms = results_filtered.shape[0]


print('Total number of ALLEGED pump and dumps:',a_npumps)
print('Total number of ALLEGED (filtered) pump and dumps:',f_a_npumps)
print('Total number of pump and dumps:',npumps)
print('Total number of (filtered) pump and dumps:',f_npumps)
print('True Pump Rate %:',true_pump_rate)
print('Total number of symbols:',nsyms)
print('Total number of filtered symbols:',f_nsyms)
print('Total unique symbols:',usyms)
print('Pumps per symbol',npumps/nsyms)
print('Pumps per (filtered) symbol',f_npumps/f_nsyms)

exchanges = ['binance','bittrex','kraken','kucoin','lbank']
syms_per = []

for e in exchanges:
    syms_per.append(result.loc[e,'Symbol'].shape[0])
    
f_syms_per = []

for e in exchanges:
    f_syms_per.append(results_filtered.loc[e,'Symbol'].shape[0])

# creates the main df broken down by exchange, with num. pumps column    
adf_2 = get_all_pumps_filtered(result,results_filtered,exchanges,'Pump and Dumps') 

adf_2['Num. Symbols'] = syms_per
adf_2['Num. Symbols (Filtered)'] = f_syms_per
adf_2['% of Total Pumps'] = adf_2['Num. Pumps (Filtered)'] / f_npumps * 100
adf_2['% of Total Symbols'] = adf_2['Num. Symbols'] / nsyms * 100
adf_2 = adf_2[['Num. Symbols','Num. Symbols (Filtered)','Num. Pumps','Num. Pumps (Filtered)','% of Total Symbols','% of Total Pumps']]
adf_2

Total number of ALLEGED pump and dumps: 2150
Total number of ALLEGED (filtered) pump and dumps: 1757
Total number of pump and dumps: 1617
Total number of (filtered) pump and dumps: 1315
True Pump Rate %: 61.16279069767442
Total number of symbols: 977
Total number of filtered symbols: 733
Total unique symbols: 737
Pumps per symbol 1.655066530194473
Pumps per (filtered) symbol 1.7939972714870396


,Num. Symbols,Num. Symbols (Filtered),Num. Pumps,Num. Pumps (Filtered),% of Total Symbols,% of Total Pumps
Exchange,,,,,,
binance,302,232,536,472,30.910952,35.893536
bittrex,242,195,580,510,24.769703,38.783270
kraken,57,8,75,12,5.834186,0.912548
kucoin,295,256,293,255,30.194473,19.391635
lbank,81,42,133,66,8.290686,5.019011


In [94]:
# For plotting the exchange breakdown
# WARNING: MEGA HARD CODED LABELS 

from bokeh.models import ColumnDataSource, FactorRange, LabelSet
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.palettes import Paired6

exchanges = ['Binance','Bittrex','Kraken','Kucoin','Lbank']=
labels = ['% of Symbols', '% of Pumps']

data = {'labels' : labels,
        '% of Symbols' : [30.9, 24.8, 5.83, 30.2, 8.29],
        '% of Pumps' : [35.9, 38.8, 0.91, 19.4, 5.02]}

x = [ (exchange, label) for exchange in exchanges for label in labels ]
counts = sum(zip(data['% of Symbols'], data['% of Pumps']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=400, title="Breakdown of Pumps by Exchange",
        tools="save",y_range=(0,100),plot_width = 700,y_axis_label="Percent")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color='white',
       fill_color=factor_cmap('x', palette=Paired6, factors=labels, start=1, end=2))

labels = LabelSet(x='x', y='counts', text='counts', level='glyph',
        x_offset=-14.5, y_offset=1, source=source, render_mode='canvas')
p.add_layout(labels)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.output_backend = "svg" # remove to go back to standard png
show(p)

In [95]:
pumps = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13']

p = figure(x_range=pumps,y_range=(0,250), plot_height=600,plot_width=700, title="Pump Count Breakdown", y_axis_label="Symbol Count"
           , x_axis_label="Number of Pumps",tools="save")

data = [224,167,132,91,50,37,20,4,2,2,2,1,0,1]

p.vbar(x=pumps, top=data, width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.output_backend = "svg" # remove to go back to standard png
show(p)

In [445]:
result.sort_values('Pump and Dumps',ascending=False)

,Symbol,Crypto Pair,Price Spikes,Volume Spikes,Alleged Pump and Dumps,Pump and Dumps,Low Market Cap
Exchange,,,,,,,
lbank,DBC/NEO,True,114,45,15,13,True
kucoin,CAPP/BTC,True,84,22,12,11,True
lbank,TKY/ETH,True,113,44,11,10,True
bittrex,DCT/BTC,True,63,24,11,10,True
bittrex,BRX/BTC,True,27,29,9,9,True
binance,MDA/ETH,True,27,26,9,9,True
bittrex,RBY/BTC,True,33,25,9,8,True
bittrex,EMC/BTC,True,32,15,8,8,True
kucoin,ADB/BTC,True,75,21,7,7,True


In [547]:
g4 = result[result['Pump and Dumps'] >= 4]
g4.head()

,Symbol,Crypto Pair,Price Spikes,Volume Spikes,Alleged Pump and Dumps,Pump and Dumps,Low Market Cap
Exchange,,,,,,,
binance,AE/BNB,True,87,8,5,5,True
binance,AE/BTC,True,88,8,6,5,True
binance,AE/ETH,True,79,9,5,5,True
binance,APPC/ETH,True,29,11,5,5,True
binance,BAT/BNB,True,22,5,5,4,True


In [241]:
# additional stuff for calculating the marketcap (removal of non-unique symbols, etc)
mcap = result[result['Market Cap'].notnull()]
mcap['RSymbol'] = mcap['Symbol'].apply(sym_split)

mcap.drop_duplicates('RSymbol',inplace=True)
mcap.sort_values(by='Market Cap',ascending=False)

totalMC = mcap['Market Cap'].sum()
mcap['% of Total Market Cap'] = (mcap['Market Cap'] / totalMC) * 100
mcap['% of Total Market Cap'] = pd.to_numeric(mcap['% of Total Market Cap'],errors='coerce')

mcap.sort_values(by='% of Total Market Cap',ascending=False)

# define low market cap as the 75th percentile and lower
low_mcap = mcap['% of Total Market Cap'].describe()['75%']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Symbol,Price Spikes,Volume Spikes,Pump and Dumps,Crypto Pair,Market Cap,% of Total Market Cap,RSymbol
Exchange,,,,,,,,
binance,BTC/USDT,1,0,0,False,1.153804e+11,4.258636e+00,BTC
binance,ETH/BTC,4,0,0,False,5.333131e+10,1.968433e+00,ETH
binance,XRP/BTC,0,2,0,False,2.129811e+10,7.861031e-01,XRP
binance,BCH/BNB,73,1,1,False,1.536841e+10,5.672406e-01,BCH
binance,EOS/BTC,96,3,3,False,9.501604e+09,3.506996e-01,EOS
binance,LTC/BNB,17,1,0,False,5.610435e+09,2.070784e-01,LTC
binance,XLM/BNB,26,4,0,False,4.372538e+09,1.613883e-01,XLM
binance,ADA/BNB,31,5,0,False,4.283152e+09,1.580891e-01,ADA
binance,TRX/BTC,74,7,3,False,3.145935e+09,1.161150e-01,TRX


In [366]:
# more helper functions

def crypto_pair(symbol,fiat_list):
    return not symbol.split('/')[1] in fiat_list

def sym_split(symbol):
    return symbol.split('/')[0]

# pulls the latest market cap information from coinmarketcap
def load_market_cap_dict():
    cmc = ccxt.coinmarketcap()
    return cmc.fetch_currencies()

# gets the market cap for one symbol from a dictionary of market cap information (ccxt coinmarketcap api)
def get_market_cap(symbol,mc_dict):
    symbol = symbol.split('/')[0] # get the base currency
    currencies = mc_dict
    if symbol in currencies.keys():
        return currencies[symbol]['info']['market_cap_usd']
    else: 
        return None

fiat = ['USD','EUR','JPY','CAD','GBP']

# requires a DF with a column name 'Symbol'
def add_crypto_pair_col(df,fiat):
    df['Crypto Pair'] = df['Symbol'].apply(crypto_pair,fiat_list=fiat)  
    
def add_market_cap(df):
    mc_dict = load_market_cap_dict()
    df['Market Cap'] = df['Symbol'].apply(get_market_cap,mc_dict=mc_dict)
    df['Market Cap'] = pd.to_numeric(df['Market Cap'],errors='coerce')
    
def add_low_mc_percent(df):
    totalMC = df['Market Cap'].sum()
    low_mcap = 0.029026851286023406
    df['% of Total Market Cap'] = (df['Market Cap'] / totalMC) * 100
    df['% of Total Market Cap'] = pd.to_numeric(df['% of Total Market Cap'],errors='coerce')
    df['Low Market Cap'] = df['% of Total Market Cap'] < low_mcap
    # FOR REMOVING THE NONES result = result[result['Market Cap'].notnull()]

def get_pumps(df,exchange_name,col_name):
    return df.loc[exchange_name,col_name].sum()

def get_all_pumps(df,exchanges,col_name):
    row_list = []
    for e in exchanges:
        total = get_pumps(df,e,col_name)
        row_list.append({'Exchange':e,'Num. Pumps':total})
        
    result = pd.DataFrame(row_list)
    result.set_index('Exchange',inplace=True)
    result.sort_index(inplace=True)
    return result